<a href="https://colab.research.google.com/github/diwert-ai/First-step-in-NLP-HSE-bootcamp-/blob/main/kaggle%20competition/inference_new_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle --upgrade
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle competitions download -c hse-nlp-bootcamp

 64% 38.0M/59.8M [00:00<00:00, 212MB/s] 
100% 59.8M/59.8M [00:00<00:00, 227MB/s]


In [ ]:
!unzip hse-nlp-bootcamp.zip

Archive:  hse-nlp-bootcamp.zip
  inflating: new_test_ml.csv         
  inflating: sample_submission.csv   
  inflating: train_ml.csv            


In [ ]:
import nltk
# nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from joblib import dump, load
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class Config:
  seed = 12536271
  samples_num = 51476
  experiment = 'new_test'

In [ ]:
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/fsnlp_hse/output/'

In [ ]:
EXPERIMENT_DIR  = OUTPUT_DIR + Config.experiment

In [ ]:
!mkdir {EXPERIMENT_DIR}

In [ ]:
def get_numeric_data(x):
    return x[['bank_id']]

def get_text_data(x):
    return x['text']

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize))
            ]))
         ])),
    ('clf', LogisticRegression(random_state=Config.seed,  warm_start=True, max_iter=1000))
])

In [ ]:
model_logreg  = load(OUTPUT_DIR+'logreg/'+'model_logreg_[sn_51476]_[sd_12536271][cv_0.7899].joblib')

In [ ]:
df = pd.read_csv('/content/train_ml.csv')
banks_list = list(df.bank.unique())
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
test = pd.read_csv('/content/new_test_ml.csv', index_col=0)
test['text'] = test.feeds.apply(lambda x: x.lower())
test['bank_id'] = test['bank'].apply(lambda x: banks_list.index(x))
test = test[['bank_id', 'text']]
test_pred = model_logreg.predict(test)
probs_lr = model_logreg.predict_proba(test)
sub = pd.DataFrame({'inds': test.index,
                    'grades': test_pred})

In [ ]:
probs_lr = model_logreg.predict_proba(test)

In [ ]:
cv_tag = '[cv_0.7899]'
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'
probs_filename = 'probs_'+suffix+'.npy'

In [ ]:
sub.to_csv(sub_filename, index=False)
np.save(probs_filename, probs_lr)

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}
!cp {probs_filename} {EXPERIMENT_DIR}

In [ ]:
!ls

 drive		        sample_submission.csv
 hse-nlp-bootcamp.zip  'sub_new test_[sn_51476]_[sd_12536271][cv_0.7899].csv'
 kaggle.json	       'sub_new_test_[sn_51476]_[sd_12536271][cv_0.7899].csv'
 new_test_ml.csv        train_ml.csv
 sample_data


In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:00<00:00, 643kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis

In [ ]:
model_logreg_pl  = load(OUTPUT_DIR+'logreg_pl/'+'model_logreg_pl_[sn_51476]_[sd_12536271][cv_0.8475].joblib')

In [ ]:
test_pred = model_logreg_pl.predict(test)

In [ ]:
sub = pd.DataFrame({'inds': test.index,
                    'grades': test_pred})
sub

In [ ]:
probs_lr_pl = model_logreg_pl.predict_proba(test)

In [ ]:
cv_tag = '[cv_0.8475]'
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'
probs_filename = 'probs_'+suffix+'.npy'

In [ ]:
sub.to_csv(sub_filename, index=False)
np.save(probs_filename, probs_lr_pl)

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}
!cp {probs_filename} {EXPERIMENT_DIR}

In [ ]:
!ls

 drive		        sample_submission.csv
 hse-nlp-bootcamp.zip  'sub_new test_[sn_51476]_[sd_12536271][cv_0.7899].csv'
 kaggle.json	       'sub_new_test_[sn_51476]_[sd_12536271][cv_0.7899].csv'
 new_test_ml.csv       'sub_new_test_[sn_51476]_[sd_12536271][cv_0.8475].csv'
 sample_data	        train_ml.csv


In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:00<00:00, 600kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis

In [ ]:
probs_avg = (probs_lr_pl + probs_lr)/2.0

In [ ]:
preds_avg=np.argmax(probs_avg, axis=1) + 1

In [ ]:
sub = pd.DataFrame({'inds': test.index,
                    'grades': preds_avg})
sub

In [ ]:
cv_tag = '[avegared]'
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'

In [ ]:
sub.to_csv(sub_filename, index=False)

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}

# AVG LB 0.79535 (m1+m2)

In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:00<00:00, 610kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis

In [ ]:
probs_avg_2 = (probs_lr_pl * probs_lr)

In [ ]:
preds_avg_2=np.argmax(probs_avg_2, axis=1) + 1

In [ ]:
sub = pd.DataFrame({'inds': test.index,
                    'grades': preds_avg_2})
sub

In [ ]:
cv_tag = '[avegared2]'
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'

In [ ]:
sub.to_csv(sub_filename, index=False)

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}

# AVG2 LB 0.79558 m1 * m2

In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {EXPERIMENT_DIR+'/sub_new_test_[sn_51476]_[sd_12536271][avegared2].csv'} -m "Submission via API"

100% 124k/124k [00:02<00:00, 50.4kB/s] 
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis